# [mtj-softtuner](https://github.com/VE-FORBRYDERNE/mtj-softtuner) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VE-FORBRYDERNE/mtj-softtuner/blob/main/mtj-softtuner.ipynb) [![Code style: black](https://img.shields.io/badge/code%20style-black-000000.svg)](https://github.com/psf/black)
### (Unofficial Mesh Transformer JAX soft-tuning notebook)

Create, in Colab, soft prompts compatible with [KoboldAI](https://github.com/KoboldAI/KoboldAI-Client) and [mkultra](https://github.com/corolla-johnson/mkultra) for your favourite GPT-J-6B-based or GPT-Neo-2.7B-based model!

See this paper https://arxiv.org/pdf/2104.08691.pdf for more information about what a soft prompt is.

---

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0). Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

<br/><br/><br/><br/>

---

<br/><br/><br/><br/>

# 1. Install and set up dependencies

## If you, at any point, restart your Colab instance by using Runtime > Restart Runtime, you will have to run this cell again.

In [ ]:
# @markdown Run this cell to do some basic setup.

!git clone https://github.com/ve-forbryderne/mtj-softtuner
!bash mtj-softtuner/install.sh

import mtj_softtuner

import os
import termcolor
import google.colab
import numpy as np

trainer = mtj_softtuner.BasicTrainer(1729, quiet=True)


def spform_callback(form_input: str):
    """
    This function gets called when we click the Submit button in the cell
    that asks you for your initial soft prompt
    """
    max_tokenized_len = trainer.data.params["max_batch_size"] - 1
    global initial_softprompt, step, starting_step, soft_in_dim
    if trainer.data.newlinemode == "s":
        form_input = form_input.replace("\n", "</s>")
    tokenizer = trainer.get_tokenizer()
    tokenized_input = tokenizer.encode(form_input, max_length=int(2e9), truncation=True)
    if len(tokenized_input) == 0:
        initial_softprompt = soft_in_dim = None
        del initial_softprompt
        del soft_in_dim
        starting_step = step = -1
        return "ERROR:  Your initial soft prompt cannot be empty!"
    if len(tokenized_input) >= max_tokenized_len:
        initial_softprompt = soft_in_dim = None
        del initial_softprompt
        del soft_in_dim
        starting_step = step = -1
        return f"ERROR:  Your initial soft prompt is too long!<br/>It is {len(tokenized_input)} tokens long,<br/>more than the maximum of {max_tokenized_len}."
    initial_softprompt = tokenized_input
    starting_step = step = 0
    soft_in_dim = len(tokenized_input)
    trainer.data.initial_softprompt = initial_softprompt
    return f"Initial soft prompt set successfully!<br/>({len(tokenized_input)} token{'' if len(tokenized_input) == 1 else 's'} long)"


google.colab.output.register_callback("spform_callback", spform_callback)

print(termcolor.colored("\n\nDone.\n\n", "green"), flush=True)

<br/><br/><br/><br/>

---

<br/><br/><br/><br/>

# 2. Download the model or log in to Google Drive

First we have to download and extract the model into your Colab instance, if you don't already have the model *uncompressed* in your Google Drive and it isn't on huggingface.co. If you do have it uncompressed in your Google Drive or the model is on huggingface.co, you can skip this cell.

You might want to look at the rest of the notebook while the model is downloading/extracting.

In [ ]:
# Feel free to modify this cell to download a finetuned GPT-J-6B model instead.
# You can also use GPT-Neo-2.7B models after first converting them with this
# notebook:
# https://colab.research.google.com/github/VE-FORBRYDERNE/mesh-transformer-jax/blob/modelcompat/convert_neo_pytorch_model_to_jax.ipynb


print(termcolor.colored("Installing pv and zstd...", "magenta"))
# The official version of pv doesn't work in Colab anymore for some reason.
# This fork contains a small patch to address the issue.
!git clone https://github.com/VE-FORBRYDERNE/pv
%cd pv
!./configure
!make
!make install
%cd ..
!apt install zstd

print(
    termcolor.colored(
        "\nDownloading GPT-J-6B model into your Colab instance...", "magenta"
    )
)
!wget -c https://mystic.the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
print(termcolor.colored("\nExtracting the model...", "magenta"))
!pv step_383500_slim.tar.zstd | tar -I zstd -x
!rm step_383500_slim.tar.zstd

print(termcolor.colored("\nDone.\n\n", "green"))

What type of pretrained model did you load? Use the dropdown below to select your model type and then run the cell below.

In [ ]:
model_type = "GPT-J-6B"  # @param ["GPT-J-6B", "GPT-Neo-2.7B", "GPT-Neo-1.3B", "fairseq-dense-13B", "fairseq-dense-6.7B", "fairseq-dense-2.7B", "fairseq-dense-1.3B", "fairseq-dense-760M", "fairseq-dense-355M", "fairseq-dense-125M"]
trainer.set_params(model_type)
print("OK.")

If your model is stored uncompressed in your Google Drive, you must run this cell below to allow us access to your Google Drive.

In [ ]:
google.colab.drive.mount("/content/drive/")

Type the path to the extracted model or huggingface.co model ID (e.g. `KoboldAI/fairseq-dense-13B`) below and then run the cell below.

If you just downloaded the normal GPT-J-6B model, then the default path that's already shown, `/content/step_383500`, is correct, so you just have to run the cell without changing the path.

If you downloaded a finetuned model, you probably know where it is stored.

If your model is in Google Drive, prefix your path with `/content/drive/MyDrive`. For example, if your model were stored in a directory in the root directory of your Google Drive called "MLP", the path would be `/content/drive/MyDrive/MLP`.

In [ ]:
ckpt_path = "/content/step_383500"  # @param {type:"string"}
trainer.data.ckpt_path = ckpt_path
print("OK.")

<br/><br/><br/><br/>

---

<br/><br/><br/><br/>

# 3. Set up soft-tuning hyperparameters and training data

If you want to save your soft prompt into your Google Drive, run this cell to login to Google Drive.

In [ ]:
google.colab.drive.mount("/content/drive/")

If you want to begin a new soft-tuning run, choose the path where we will save to. You will see a file there before the soft-tuning process is fully complete, it will be there so you can resume the soft-tuning process later if your Colab instance crashes. If you want to save into your Google Drive, prefix your path with `/content/drive/MyDrive`.

If you want to resume a soft-tuning run for the aforementioned reason, choose the path to an existing MTJSP file.

In [ ]:
save_file = "/content/drive/MyDrive/my_softprompt.mtjsp"  # @param {type:"string"}

save_file = save_file.replace("\\", "/")
if save_file.endswith("/"):
    trainer.raise_configuration_error("save_file should be a file, not a directory.")

os.makedirs(save_file.rsplit("/", 1)[0].strip(), exist_ok=True)

if os.path.exists(save_file):
    npz = np.load(save_file, allow_pickle=True)
    step = np.uint32(npz["step"]).item()
    print("OK.")
    print(f"We will resume soft-tuning at step {step + 1}.")
else:
    print("OK.")
    print("We will begin a new soft-tuning run.")

trainer.data.save_file = save_file

If you are beginning a new soft-tuning run, choose a string to initialize your soft prompt with. It should be roughly 20-200 tokens long. The maximum allowed length is 449 tokens if you're using fairseq-dense-13B, or 2047 tokens otherwise.. It's recommended that your string should end with two newline characters and have no other leading or trailing whitespace on any line.

If you're resuming a soft-tuning run with an existing MTJSP file, you can skip this cell.

In [ ]:
#@markdown ### Run this cell once to make a text box appear for you to type in your initial soft prompt.<br/>After that, press the "Submit" button underneath the text box; do not run this cell a second time.
%%html
<form>
    <textarea id="softprompt" rows="10" cols="80">Le Jeu du Prochain Train itself is simplicity in motion. The object: Be the last of your round's six to jump from one side of the tracks to the other - that is, across the tracks - before the train passes.

</textarea>
    <br/>
    <p><input id="submit-softprompt" type="button" value="Submit" /></p>
</form>
<br/>
<p id="softprompt-message"></p>
<script type="text/javascript">
    (function() {
        var submit = document.getElementById("submit-softprompt");
        var softprompt = document.getElementById("softprompt");
        var message = document.getElementById("softprompt-message");
        submit.addEventListener("click", async function() {
            var msg = await google.colab.kernel.invokeFunction("spform_callback", [softprompt.value], {});
            message.innerHTML = msg.data['text/plain'].replace(/(?:^ *'*)|(?:'* *$)/g, "");
        });
        softprompt.addEventListener("input", function() {
            message.innerHTML = "";
        });
    })();
</script>

If your dataset is stored in Google Drive, you have to log in to Google Drive so we can access it.

In [ ]:
google.colab.drive.mount("/content/drive/")

If your dataset is a single txt file or collection of txt files, we have to convert it to npy format first. If you have already used this notebook to convert your txt dataset to npy, you can skip this cell.

`dataset_path` should be the path to either a single txt file or a folder with one or more txt files in it. Then run the cell, and we will make a npy file using your dataset at the given path (we will create the required directory tree for the output file if the output file's directory doesn't already exist). If your txt files are in Google Drive, prefix your path with `/content/drive/MyDrive`.

`batch_size` is explained in this article: https://medium.com/mini-distill/effect-of-batch-size-on-training-dynamics-21c14f7a716e. The maximum possible batch size is 2048 minus the number of tokens in your initial soft prompt, so if your initial soft prompt is 49 tokens long then the maximum allowed batch size is 1999. If your batch_size is too high, we will automatically lower it to the highest possible value, so just leave it at 2048 if you want us to do that. If you're tuning fairseq-dense-13B, the maximum possible batch size is instead 450 minus the number of tokens in your initial soft prompt.

Epochs is the amount of times to repeat your dataset (it will be shuffled every time).

In [ ]:
dataset_path = "/content/drive/MyDrive/dataset.txt"  # @param {type:"string"}
output_file = "/content/drive/MyDrive/output.npy"  # @param {type:"string"}
batch_size = 2048  # @param {type:"integer"}
epochs = 1  # @param {type:"integer"}
trainer.tokenize_dataset(dataset_path, output_file, batch_size, epochs)
print("OK.")

Here, we set the npy that we will use for soft-tuning.

`gradient_accumulation_steps` is described here: https://towardsdatascience.com/what-is-gradient-accumulation-in-deep-learning-ec034122cfa. It's preferable to have gradient accumulation steps in the 16-32 range.

In [ ]:
dataset_file = "/content/drive/MyDrive/output.npy"  # @param {type:"string"}
gradient_accumulation_steps = 16  # @param {type:"integer"}
trainer.data.dataset_file = dataset_file
trainer.data.gradient_accumulation_steps = gradient_accumulation_steps
print("OK.")

Now it is time to set the other soft-tuning hyperparameters. Edit the numbers below (or don't edit any) and then run the cell.

By default we use the same modified version of the Adam optimization algorithm that Mesh Transformer JAX uses by default for training.

The main thing you have to pay attention to is `lr` and `max_grad_norm`; everything else is basically universally OK for training. It is recommended to set `lr` to somewhere in the `1e-5` to `5e-5` range. Higher `lr` results in the trainer having a stronger effect. Values higher than around `7e-5` tend to result in exploding gradients (numerical instability) and should be avoided! You can tell when this happens because the Gradient L2 Norm will start increasing abnormally and eventually "explode" to values in the thousands. If the trainer still isn't strong enough, you should train with more epochs instead of risking the numerical instability.

`max_grad_norm` controls the maximum allowed rate at which the soft prompt can be trained, so if the trainer tries to change the soft prompt by too much in one step, the changes will be scaled down uniformly. Lower values are more restrictive.

* `save_every` (`int` > 0): We'll save an MTJSP file every this many steps so that if you are disconnected from Colab, you can continue from an earlier point in the training.
* `warmup` (`float` between 0.0 and 1.0 inclusive): What portion of the beginning of the total training steps should be warmup steps. The learning rate for warmup steps starts at 0.0 and increases linearly to the maximum learning rate.
* `lr` (`float` > 0): Aforementioned maximum learning rate.
* `end_lr_multiplier` (`float` > 0): After the warmup steps, the remaining training steps have a learning rate controlled by a cosine function that goes from the maximum learning rate to this proportion of the maximum learning rate (i.e. the default is one-tenth of the maximum learning rate).
* `weight_decay` (`float` between 0.0 and 1.0 inclusive): The soft prompt you're going to train is actually a two-dimensional array of floating-point numbers. The absolute values of the numbers tend to be pretty small (less than 1). If the absolute value of one of those numbers grows too large, the trainer tends to try to reduce the absolute values of the entire array, resulting in the entire array being filled with zeros and ruining your soft prompt. This setting effectively restricts the absolute value of the numbers in the array (higher weight decay value means the maximum absolute value is lower) to help prevent this scenario. Of course, setting it too high would lower the absolute value of your array, too, just like what would happen if you'd set it too low, so there's usually an optimal value. People have suggested 0.1 as a good all-around weight decay factor.
* `max_grad_norm` (`float` > 0): Controls the maximum allowed rate at which the soft prompt can be trained, so if the trainer tries to change the soft prompt by too much in one step, the changes will be scaled down uniformly. Lower values are more restrictive. If the "Gradient L2 Norm" is much higher (e.g. at least twice as much) on average than this value, you should probably raise this value.

In [ ]:
lr = 3e-5  # @param {type:"number"}
max_grad_norm = 10.0  # @param {type:"number"}
weight_decay = 0.1  # @param {type:"number"}
warmup = 0.1  # @param {type:"number"}
end_lr_multiplier = 0.1  # @param {type:"number"}
save_every = 50  # @param {type:"integer"}
trainer.data.stparams = {
    param: globals()[param]
    for param in (
        "lr",
        "max_grad_norm",
        "weight_decay",
        "warmup",
        "end_lr_multiplier",
        "save_every",
    )
}
print("OK.")

<br/><br/><br/><br/>

---

<br/><br/><br/><br/>

# 4. Soft-tune the model

## If you reached here and at any point after that restarted your Colab instance by using Runtime > Restart Runtime, you will have to run the step 1 cell again and then run the cells below again. You don't have to re-run any cells in between.

In [ ]:
# @markdown This can take quite a while depending on how fast your Colab instance is. Note that currently (2021-10-12), all Colab TPU instances will train at the same speed, some just take longer to initialize the model than others. Sometimes this cell may throw an error about "deadline exceeded", in which case you should restart your Colab instance (Runtime > Restart Runtime), run Step 1 again and then run this cell again.

trainer.train()

Use this cell to login to Google Drive if required.

In [ ]:
google.colab.drive.mount("/content/drive/")

Once you finish soft-tuning, you can use the following cell to convert your MTJSP file to a [KoboldAI](https://github.com/KoboldAI/KoboldAI-Client)-compatible ZIP file:

(`supported` should be the name of a model or a comma-separated list of such names that the soft prompt is intended to be used with)

In [ ]:
output_file = "/content/drive/MyDrive/my_softprompt.zip"  # @param {type:"string"}
name = "Untitled"  # @param {type:"string"}
author = ""  # @param {type:"string"}
supported = "Generic 6B"  # @param {type:"string"}
description = "Baby shoes"  # @param {type:"string"}
trainer.export_to_kobold(output_file, name, author, supported, description)

Or this one, to convert your MTJSP file to an mkultra-compatible JSON file.

In [ ]:
output_file = "/content/drive/MyDrive/my_softprompt.json"  # @param {type:"string"}
soft_prompt_name = "Untitled"  # @param {type:"string"}
soft_prompt_description = "Baby shoes"  # @param {type:"string"}
trainer.export_to_kobold(output_file, soft_prompt_name, soft_prompt_description)